## Import Libraries

In [1]:
# import numpy for matrix operation
import numpy as np

# Importing Gensim
import gensim
from gensim import corpora

# to generate dictionary of unique tokens
from gensim.corpora import Dictionary

#LDA model
from gensim.models import LdaModel, LdaMulticore

# to suppress warnings
from warnings import filterwarnings
filterwarnings('ignore')

#plotting tools
import matplotlib.pyplot as plt


#To extract my sequences from myfile.phy
import phylip


import os
import itertools



merging= True
DEBUG = False


if DEBUG:
    #To print progress of the training procedure on screen
    import logging
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.NOTSET)

## Tokenize Documents

Tokenization: Split the text into sentences and the sentences into words.

Tokenization is used in natural language processing to split paragraphs and sentences into smaller units that can be more easily assigned meaning.


In [2]:
def tokenize(seq_list, k, gap_type='None', kmer='overlap'):    
   
    if gap_type == 'seq':
        seq_list = [seq.replace('-', '') for seq in seq_list]
        
    elif gap_type == 'col':
        # String List to Column Character Matrix Using zip() + map()
        seq_matrix = np.transpose(list(map(list, zip(*seq_list))))      
        #remove columns if the column has an element '-'
        seq_matrix_cleaned = np.array([col for col in seq_matrix.T if '-' not in col]).T
        #Convert List of lists to list of Strings again
        seq_list = [''.join(ele) for ele in seq_matrix_cleaned] 
        
    if DEBUG:
        np.savetxt ('seq_list', seq_list,  fmt='%s')     
    
    #docs: list of lists of each document kemers
    docs = []  
    if kmer == 'overlap':
        for seq in seq_list:       
            doc=[]     
            for i in range(len(seq) - k + 1):
                kmer = seq[i:i+k]
                doc.append(kmer)
            docs.append(doc) 
            
    elif kmer == 'not_overlap':
        for seq in seq_list:   
            doc = [seq[i:i+k] for i in range(0, len(seq), k)]
            docs.append(doc) 
    return docs



In [3]:
current = os.getcwd()
print(current) 

topics_loci=[]
num_loci=19

for i in range(num_loci):
    print(f'\n============= locus {i} =============\n')
    
    #================= Extract labels and sequences ==================
    locus_i = 'locus'+str(i)+'.txt'
    locus = os.path.join(current,'loci',locus_i)
    
    label,sequence,varsites = phylip.readData(locus, type='NotSTANDARD')
    #print(f"label = {label}")
    #print(f"sequence = {sequence}")    
    
    #========================= Extract k-mers ========================
    docs=[]
    for k in range(2,10,2):    
#         tokenize_k = tokenize(sequence, k )       #"with gap" and "overlapped k-mers"
        tokenize_k = tokenize(sequence, k, gap_type='seq', kmer = 'not_overlap')     #"No gaps'-'" in each sequence and "No overlapped k-mers"
#         tokenize_k = tokenize(sequence, k, kmer = 'not_overlap' )    #"with gap" and "No overlapped k-mers"
        
        if len(docs)>0:
            docs =[docs[i]+tokenize_k[i] for i in range(len(tokenize_k))]
        else:
            docs = tokenize_k  
    #count number of all words in docs
    count = 0
    for doc in docs:
        count += len(doc)   
    print(f"Test ===> Number of all words in docs = {count}")
    
    
    #====================== k-mers after merging =====================
    if merging:
        letters = ['sca','sce','sct','smc', 'smi', 'sms']
        merging_nums=[]
        for item in letters:
            merg_indxs= [label.index(i) for i in label if item in i]
            merging_nums.append(merg_indxs)
            
        merging_nums=[len(i) for i in merging_nums]
        print(f"merging_nums = {merging_nums}")
        
        docs_merged=[]
        j=0
        for num in merging_nums:
            doc_merged = list(itertools.chain.from_iterable(docs[j:j+num])) 
            docs_merged.append(doc_merged)
            j +=num

        #count number of all words in docs
        count = 0
        for doc in docs_merged:
            count += len(doc)   
        print(f"Test ===> Number of all words in docs_merged = {count}") 

    #=================== Dictionary of Unique Tokens ==================
    docs = docs_merged
    dictionary = Dictionary(docs)
    print(f"\nDictionary:\n{dictionary}")
    
    #============ Filtering: remove rare and common tokens ============
    dictionary.filter_extremes(no_below=2, no_above=0.5)
    print(f"\nDictionary after filtering:\n{dictionary}")
    
    #================== Vectorize data: Bag-of-words ==================
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    print(f'\nNumber of documents: {len(corpus)}')
    print(f'Number of unique tokens: {len(dictionary)}')
    
    #======================= LDA Model: Training ======================
    # Set training parameters.
    num_topics = 5    
    chunksize = 20    
    passes = 50     
    iterations = 1000  
    eval_every = 1

    # Make a index to word dictionary.
    temp = dictionary[0]  
    id2word = dictionary.id2token

    model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha=1, eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every, minimum_probability=0, update_every=5 )

    #================== Print topics/words frequencies ================
    if DEBUG:
        for idx, topic in model.print_topics(-1):
            print("Topic: {} \nWords: {}\n".format(idx, topic))
            
    #=============== Assigning the topics to the documents ============
    docs_tuples = model[corpus]

#     for num, doc_i in enumerate(docs_tuples):
#         print(f"doc {num}     {doc_i}")
            
    #==================== topics list for current locus ===============            
    delta_type='dirichlet'
    topics = []
    for num, doc in enumerate(docs_tuples):
        first=[]
        second=[]
        for tuple_i in doc:
            first.append(tuple_i[0])
            second.append(tuple_i[1])
        
        if delta_type == 'equal':
            sum_frequencies = sum(second)
            res = 1- sum_frequencies
            delta = (np.ones(num_topics-len(doc)))*(res/(num_topics-len(doc)))
            
        elif delta_type == 'dirichlet':
            sum_frequencies = sum(second)
            res = 1- sum_frequencies
            delta = ((np.random.dirichlet(np.ones(num_topics-len(doc)),size=1))*res).flatten()

        topics_freq=[]
        j=0
        for i in range(num_topics):
            if i in first:
                topics_freq.append(second[first.index(i)])
            else:
                topics_freq.append(delta[j])
                j +=1
        topics.append(topics_freq)
        
    #print(f'\nTopics :\n{topics}')
    topics_loci.append(topics) 
    





/Users/tara/Desktop/TPContml_project

============= locus 0 =============

Test ===> Number of all words in docs = 33360
merging_nums = [18, 8, 10, 2, 6, 4]
Test ===> Number of all words in docs_merged = 33360

Dictionary:
Dictionary(581 unique tokens: ['A', 'AA', 'AAAA', 'AAAAAC', 'AAAATGAT']...)

Dictionary after filtering:
Dictionary(300 unique tokens: ['A', 'AAACCCTC', 'AACTAA', 'AACTAATG', 'AACTAT']...)

Number of documents: 6
Number of unique tokens: 300

============= locus 1 =============

Test ===> Number of all words in docs = 26064
merging_nums = [18, 8, 10, 2, 6, 4]
Test ===> Number of all words in docs_merged = 26064

Dictionary:
Dictionary(370 unique tokens: ['??', '????', '?????TTA', '?T', '?TTA']...)

Dictionary after filtering:
Dictionary(91 unique tokens: ['AAAAGT', 'AAAAGTAT', 'AAAT', 'AAATAC', 'AAATACAC']...)

Number of documents: 6
Number of unique tokens: 91

============= locus 2 =============

Test ===> Number of all words in docs = 21024
merging_nums = [18, 8, 

In [4]:
letters = ['sca','sce','sct','smc', 'smi', 'sms']

def infile(topics_loci_concatenated, lletters):     
    num_pop= len(topics_loci_concatenated)
    with  open("infile", "w") as f:   
        f.write('     {}    {}\n'.format(num_pop, num_loci)) 
        f.write('{} '.format(num_topics)*num_loci) 
        f.write('\n') 
        for i in range(num_pop):
            #myname="L"+str(i)
            myname = lletters[i]
            f.write(f'{myname}{" "*(11-len(myname))}{" ".join(map(str, topics_loci_concatenated[i]))}\n')            
    f.close()

        

def run_contml(infile): 
    os.system('rm outfile outtree')
    os.system('echo "y" | /Users/tara/bin/contml') 
    
    #read the outtree file
    with open('outtree', 'r') as f:
        tree = f.read().replace('\n', ' ')
    return tree





print(f'\n========== Write results in "infile" to use in CONTML ======= \n')

#for each locus remove last column from topic matrix
topics_loci_missingLast = [[item[i][:-1] for i in range(len(item))] for item in topics_loci]


#concatenation of the topics for al loci
topics_loci_concatenated = topics_loci_missingLast[0]
for i in range(1,len(topics_loci_missingLast)):
    topics_loci_concatenated = [a+b for a, b in zip(topics_loci_concatenated, topics_loci_missingLast[i]) ]
    
#print(f'topics_loci_concatenated =\n{topics_loci_concatenated}')



#generate infile
infile(topics_loci_concatenated, letters)


#run CONTML
ourtree = run_contml(infile)
print(ourtree)


#Figtree
os.system("/Users/tara/bin/figtree outtree")


========== Write results in "infile" to use in CONTML ======= 



Continuous character Maximum Likelihood method version 3.697

Settings for this run:
  U                       Search for best tree?  Yes
  C  Gene frequencies or continuous characters?  Gene frequencies
  A   Input file has all alleles at each locus?  No, one allele missing at each
  O                              Outgroup root?  No, use as outgroup species 1
  G                      Global rearrangements?  No
  J           Randomize input order of species?  No. Use input order
  M                 Analyze multiple data sets?  No
  0         Terminal type (IBM PC, ANSI, none)?  ANSI
  1          Print out the data at start of run  No
  2        Print indications of progress of run  Yes
  3                              Print out tree  Yes
  4             Write out trees onto tree file?  Yes

  Y to accept these or type the letter for one to change
Adding species:
   1. sca       
   2. sce       
   3. sct       
   4. s

rm: outfile: No such file or directory
rm: outtree: No such file or directory


javax.swing.UIManager$LookAndFeelInfo[Metal javax.swing.plaf.metal.MetalLookAndFeel]
javax.swing.UIManager$LookAndFeelInfo[Nimbus javax.swing.plaf.nimbus.NimbusLookAndFeel]
javax.swing.UIManager$LookAndFeelInfo[CDE/Motif com.sun.java.swing.plaf.motif.MotifLookAndFeel]
javax.swing.UIManager$LookAndFeelInfo[Mac OS X com.apple.laf.AquaLookAndFeel]


0